# Hybrid NF-MCMC Demo - Algorithm 1: Pre-trainging NF
### Run script to run example Algorithm 1 experiment for dV = 0

Total script run time ~30 minutes

Imports

In [ ]:
import torch
import numpy as np
from matplotlib import pyplot as plt
from tqdm import tqdm, trange
from scipy.spatial.distance import squareform, cdist
import pandas as pd
import logging
import sys
import os
import json
from demo_utils import get_project_root
# Add the custom path for hybrid_nf_mcmc
project_root = get_project_root()
hybrid_nf_mcmc_path = os.path.join(project_root, "hybrid_NF_MCMC")
sys.path.append(hybrid_nf_mcmc_path)
from utils import (get_project_root, setup_logger, get_dataloader, calculate_well_statistics, 
                  set_icl_color_cycle, get_icl_heatmap_cmap, plot_loss, 
                  plot_frequency_heatmap, generate_samples, 
                  calculate_pair_correlation, plot_pair_correlation, save_rdf_data, plot_acceptance_rate,
                  plot_avg_free_energy, plot_well_statistics, plot_avg_x_coordinate,
                  plot_multiple_avg_x_coordinates)
import argparse
import csv

set_icl_color_cycle()
cmap_div = get_icl_heatmap_cmap("diverging")

# importing the NF and MC modules
project_root = get_project_root()
nf_path = os.path.join(project_root, "NF")
sys.path.append(nf_path)
sys.path.append(project_root)
import normflows as NF
import MCMC as MC

Parameter inputs

To increase accuracy and precision run for the number of training cycles and with number of parallel MCMC runs

In [3]:
experiment_name = "A1_demo"

# EXPERIMENT VARIABLES
NUM_MC_RUNS = 10       # how many mc run running in 'parallel'
MASTER_SEED = 42        # master seed to give seeds for each mc run to run differently
NUM_PARTICLES = 3
NUM_DIM = 2
NUM_TRAINING_CYCLES = 0

# mc needed params
TEMP = 1.0
RHO = 0.03
ASPECT_RATIO = 1.0
VISUALISE = True
CHECKING = True
EQUILIBRATION_STEPS = 5000
NUM_WELLS = 2
V0_LIST = [-10.0,-10.0]              # depth of wells - adjust to change the dV
R0 = 1.2                
K_VAL = 15             
HALF_BOX = ((NUM_PARTICLES/RHO)**(1/NUM_DIM))/2
INITIAL_MAX_DISPLACEMENT = 0.65
SAMPLING_FREQUENCY = 150
ADJUSTING_FREQUENCY = 5000

# nf needed params
INITIAL_TRAINING_NUM_SAMPLES = 10240
BATCH_SIZE = 512
K = 15
EPOCHS = 20
LR = 1e-4
WEIGHT_DECAY = 0
UPDATE_NUM_SAMPLES = 0
HIDDEN_UNITS = 256
NUM_BINS = 32
N_BLOCKS = 8

# post training runs parameters
TESTING_SAMPLING_FREQUENCY = 10
TESTING = True
BIG_MOVE_ATTEMPTS = 20
BIG_MOVE_INTERVAL = 100
NUM_SAMPLES_FOR_ANALYSIS = BIG_MOVE_ATTEMPTS * NUM_MC_RUNS

Saving parameters to a JSON file, setting up directory and logging

In [ ]:
directory = os.path.join(project_root, "demos/data", experiment_name)
if not os.path.exists(directory):
    os.makedirs(directory)
# Setup a global experiment logger that logs to a file in the experiment directory.
experiment_log_file = os.path.join(directory, "experiment.log")
experiment_logger = setup_logger("experiment", experiment_log_file, file_level=logging.DEBUG, stream_level=logging.INFO)
experiment_logger.info("half box is: " + str(HALF_BOX))
experiment_logger.info(f"Directory created at: {directory}")

mc_runs_directory = os.path.join(directory, "mc_runs")
os.makedirs(mc_runs_directory, exist_ok=True)
training_rounds_directory = os.path.join(directory, "training")
os.makedirs(training_rounds_directory, exist_ok=True)

# Save experiment parameters to a JSON file in the parent experiment directory
experiment_params = {
    "NUM_MC_RUNS": NUM_MC_RUNS,
    "MASTER_SEED": MASTER_SEED,
    "NUM_PARTICLES": NUM_PARTICLES,
    "NUM_DIM": NUM_DIM,
    "NUM_TRAINING_CYCLES": NUM_TRAINING_CYCLES,
    "INITIAL_TRAINING_NUM_SAMPLES": INITIAL_TRAINING_NUM_SAMPLES,
    "BATCH_SIZE": BATCH_SIZE,
    "K": K,
    "EPOCHS": EPOCHS,
    "LR": LR,
    "WEIGHT_DECAY": WEIGHT_DECAY,
    "UPDATE_NUM_SAMPLES": UPDATE_NUM_SAMPLES,
    "HIDDEN_UNITS": HIDDEN_UNITS,
    "NUM_BINS": NUM_BINS,
    "N_BLOCKS": N_BLOCKS,
    "TEMP": TEMP,
    "RHO": RHO,
    "ASPECT_RATIO": ASPECT_RATIO,
    "VISUALISE": VISUALISE,
    "CHECKING": CHECKING,
    "EQUILIBRATION_STEPS": EQUILIBRATION_STEPS,
    "NUM_WELLS": NUM_WELLS,
    "V0_LIST": V0_LIST,
    "R0": R0,
    "K_VAL": K_VAL,
    "HALF_BOX": HALF_BOX,
    "INITIAL_MAX_DISPLACEMENT": INITIAL_MAX_DISPLACEMENT,
    "SAMPLING_FREQUENCY": SAMPLING_FREQUENCY,
    "ADJUSTING_FREQUENCY": ADJUSTING_FREQUENCY,
    "TESTING": TESTING,
    "TESTING_SAMPLING_FREQUENCY": TESTING_SAMPLING_FREQUENCY,
    "BIG_MOVE_ATTEMPTS": BIG_MOVE_ATTEMPTS,
    "BIG_MOVE_INTERVAL": BIG_MOVE_INTERVAL,
    "NUM_SAMPLES_FOR_ANALYSIS": NUM_SAMPLES_FOR_ANALYSIS
}
json_file_path = os.path.join(directory, "params.json")
with open(json_file_path, "w") as f:
    json.dump(experiment_params, f, indent=4)
experiment_logger.info(f"Experiment parameters saved to {json_file_path}")

Initialisation and data collection: initialise MCMC runs, equilibrate them and collect samples for NF pre-training

Run-time: ~5 minutes

In [ ]:
# initialisation of experiment - sets up the monte carlo runs and equilibrates and collects samples
mc_runs = []  # list to store each Monte Carlo simulation instance
for i in range(NUM_MC_RUNS):
    seed = i + MASTER_SEED
    np.random.seed(seed)

    # Create a dedicated folder and logger for this Monte Carlo run.
    run_dir = os.path.join(mc_runs_directory, f"run_{i+1:03d}")
    os.makedirs(run_dir, exist_ok=True)
    mc_log_file = os.path.join(run_dir, "mc_run.log")
    mc_logger = setup_logger(f"MC_run_{i+1:03d}", mc_log_file, file_level=logging.DEBUG, stream_level=logging.WARNING)

    # even i runs initialise left and odd initialise right
    if i % 2 == 0:
        particles, sim_box = MC.initialise_low_left(
                num_particles=NUM_PARTICLES,
                rho=RHO,
                aspect_ratio=ASPECT_RATIO,
                visualise=False,
                checking=False
            )
        experiment_logger.info(f"run {i} starts in left")
    else:
        particles, sim_box = MC.initialise_low_right(
                num_particles=NUM_PARTICLES,
                rho=RHO,
                aspect_ratio=ASPECT_RATIO,
                visualise=False,
                checking=False
            )
        experiment_logger.info(f"run {i} starts in right")
        
    mc_run = MC.MonteCarlo(
        particles=particles,
        sim_box=sim_box,
        temperature=TEMP,
        num_particles=NUM_PARTICLES,
        num_wells=NUM_WELLS,    # Pass number of wells
        V0_list=V0_LIST,                  # Pass well depth
        r0=R0,
        k=K_VAL,
        initial_max_displacement=INITIAL_MAX_DISPLACEMENT,  # Adjust as needed
        target_acceptance=0.5,
        timing=False,
        checking=CHECKING,
        logger=mc_logger,       # Each run now logs to its own file
        seed=seed
    )

    mc_runs.append(mc_run)
    experiment_logger.info(f"Monte Carlo run {i} initialised and stored")

# Plot the potential wells for visualization
MC.visualise.plot_potential(
    box_siz_x=sim_box.box_size_x,
    box_size_y=sim_box.box_size_y,
    V0_list=V0_LIST,
    r0=R0,
    k=K_VAL,
    num_wells=NUM_WELLS,
    output_path=directory
)

experiment_logger.info("All Monte Carlo runs initialised")
experiment_logger.info("list of run instances: " + str(mc_runs))

# equilibration of each mc run
for mc_run in mc_runs:
    for step in range(1, EQUILIBRATION_STEPS + 1):
        mc_run.particle_displacement()
        if step % ADJUSTING_FREQUENCY == 0:
            mc_run.adjust_displacement()
        if step % SAMPLING_FREQUENCY == 0:
            sample = mc_run.sample(step)
            mc_run.local_samples.append(sample)
    
# Plot and save the equilibrated configuration for each MC run
for i, mc_run in enumerate(mc_runs):
    run_dir = os.path.join(mc_runs_directory, f"run_{i+1:03d}")
    fig = plt.figure(figsize=(8, 6))
    current_config = mc_run.particles
    plt.scatter(current_config[:, 0], current_config[:, 1], alpha=0.6)
    plt.xlim(0, 2*HALF_BOX)
    plt.ylim(0, 2*HALF_BOX)
    plt.xlabel(r'$x$')
    plt.ylabel(r'$y$') 
    plt.title(f'Equilibrated Configuration - MC Run {i+1}')
    
    # Save plot to the MC run's directory
    config_plot_path = os.path.join(run_dir, f"equilibrated_config.png")
    fig.savefig(config_plot_path, bbox_inches='tight')
    plt.close(fig)
    
    mc_run.logger.info(f"Equilibrated configuration plot saved to: {config_plot_path}")

# Initialize MCMC step counter right after equilibration
total_mcmc_steps = 0

# Initialize acceptance rate tracking at the very beginning
p_acc_history = [0.0]  # Initial acceptance rate is 0
mcmc_steps_history = [0]  # Start from MCMC step 0
big_move_attempts = 0
big_move_accepts = 0


# collect initial set of training samples
global_samples_mc = []  # storage of mc samples in mc box range of 0,10
production_runs = int(INITIAL_TRAINING_NUM_SAMPLES/(NUM_MC_RUNS / SAMPLING_FREQUENCY))
experiment_logger.info("production runs per cycle: " + str(production_runs))
for mc_run in mc_runs:
    for step in range(1, production_runs + 1):
        mc_run.particle_displacement()
        # total_mcmc_steps += 1  # Count each MCMC step
        if step % SAMPLING_FREQUENCY == 0:
            sample = mc_run.sample(step)
            mc_run.local_samples.append(sample)
            global_samples_mc.append(sample[6])  # only config being stored (in MC box bound)

global_samples_nf = np.array([np.array([particle - np.array([HALF_BOX, HALF_BOX]) for particle in config]) for config in global_samples_mc])


NF pre-training

Run-time: ~25 minutes on a M1 chip (no CUDA)

In [ ]:
# prepping initial samples as for dataloader
enable_cuda = True
device = torch.device('cuda' if torch.cuda.is_available() and enable_cuda else 'cpu')
experiment_logger.info('device is: ' + str(device))

unique_data = np.unique(global_samples_nf, axis=0)
experiment_logger.info("Total unique samples: " + str(unique_data.shape[0]))

dataloader = get_dataloader(global_samples_nf, NUM_PARTICLES, NUM_DIM, device, batch_size=BATCH_SIZE, shuffle=True)
experiment_logger.info("DataLoader details:")
experiment_logger.info("Dataset size: " + str(len(dataloader.dataset)))
experiment_logger.info("Number of batches: " + str(len(dataloader)))
first_batch = next(iter(dataloader))
experiment_logger.info("Shape of data in first batch: " + str(first_batch[0].shape))

# initialise the normalizing flow model
bound = HALF_BOX
base = NF.Energy.UniformParticle(NUM_PARTICLES, NUM_DIM, bound, device=device)
# target = NF.Energy.SimpleLJ(NUM_PARTICLES * NUM_DIM, NUM_PARTICLES, 1, bound)    # not used for now
# K = 15 - now defined in the argument of function
flow_layers = []
for i in range(K):
    flow_layers += [NF.flows.CircularCoupledRationalQuadraticSpline(NUM_PARTICLES * NUM_DIM, NUM_BINS, HIDDEN_UNITS,
                    range(NUM_PARTICLES * NUM_DIM), num_bins=NUM_BINS, tail_bound=bound)]
model = NF.NormalizingFlow(base, flow_layers).to(device)
experiment_logger.info(f'Model prepared with {NUM_PARTICLES} particles and {NUM_DIM} dimensions!')

# Create a dedicated folder and logger for the first training round of the normalizing flow.
nf_training_dir = os.path.join(training_rounds_directory, "pre_training_results")
os.makedirs(nf_training_dir, exist_ok=True)
nf_log_file = os.path.join(nf_training_dir, "nf_training_round.log")
nf_logger = setup_logger("NF_training_round", nf_log_file, file_level=logging.DEBUG, stream_level=logging.WARNING)

# train the normalizing flow on initial training batch
loss_hist = np.array([])
loss_epoch = []

optimizer = torch.optim.Adam(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
experiment_logger.info(f'Optimizer prepared with learning rate: {LR}')
data_save = []

for it in trange(EPOCHS, desc="Training Progress"):
    epoch_loss = 0
    for batch in dataloader:
        optimizer.zero_grad()
        # Compute loss
        # loss,z = model.reverse_kld(num_samples)
        loss = model.forward_kld(batch[0].to(device))
        # Do backprop and optimizer step
        # if ~(torch.isnan(loss) | torch.isinf(loss)):
        if torch.isnan(loss) or torch.isinf(loss):
            experiment_logger.warning("Warning: Loss is NaN or Inf. Skipping this batch.")
        else:
            loss.backward()
            optimizer.step()
            # scheduler.step()
        # Log loss
        loss_hist = np.append(loss_hist, loss.to('cpu').data.numpy())
        epoch_loss += loss.item()
    loss_epoch.append(epoch_loss / len(dataloader))
    nf_logger.info(f'Epoch {it+1}/{EPOCHS}, Loss: {loss_epoch[-1]:.4f}')

# Plot loss
loss_plot_path_svg, loss_plot_path_png = plot_loss(loss_epoch, nf_training_dir)
nf_logger.info(f"Loss plot saved successfully to: {loss_plot_path_png}")

model_path = os.path.join(nf_training_dir, "pre_trained_model_circular_spline_res_dense.pth")
torch.save(model.state_dict(), model_path)
nf_logger.info(f"Model saved successfully to: {model_path}")

# change to eval mode to test the NF model
model.eval()
samples_for_mc = []
z = model.sample(NUM_MC_RUNS)
z_ = z.reshape(-1, NUM_PARTICLES, NUM_DIM)  # Reshape as required
z_ = z_.to('cpu').detach().numpy()  # Move to CPU and convert to NumPy
z_ = z_ + HALF_BOX  # Add HALF_BOX to each element in the samples
print(z_)

# Generate samples
a_ = generate_samples(model, NUM_PARTICLES, NUM_DIM, 
                     n_iterations=NUM_SAMPLES_FOR_ANALYSIS // 5000 + 1, 
                     samples_per_iteration=5000)
a_ = a_ + HALF_BOX  # Add HALF_BOX to each element in the samples
print(a_.shape)
samples_path = os.path.join(nf_training_dir, "samples.npy")
np.save(samples_path, a_)
final_model_samples = a_

# make heatmap
heatmap_path_svg, heatmap_path_png = plot_frequency_heatmap(a_- HALF_BOX, nf_training_dir, cmap_div, HALF_BOX)
nf_logger.info(f"Frequency heatmap saved successfully to: {heatmap_path_png}")

# Calculate and plot pair correlation function
r_vals, g_r = calculate_pair_correlation(a_ - HALF_BOX, NUM_PARTICLES, HALF_BOX, dr=HALF_BOX/50)
pair_corr_path_svg, pair_corr_path_png = plot_pair_correlation(
    r_vals, 
    g_r, 
    nf_training_dir
)
print(f"Pair correlation function plot saved successfully to: {pair_corr_path_svg} and {pair_corr_path_png}")



Post training hybrid NF-MCMC testing phase

Run-time: ~1 min

In [ ]:
# -------------------------
# After the full training, running the mc_runs with the samples from the final trained model
# -------------------------
trained_nf_model = model 

# For each Monte Carlo simulation instance, assign the trained model:
for mc_run in mc_runs:
    mc_run.set_nf_model(trained_nf_model)

if TESTING:
    test_configs = final_model_samples  # configurations produced by the final model for big move suggestions
    
    free_energy_array = []
    
    # Initialize counters for acceptance statistics
    nf_to_mc_attempts_total = 0
    nf_mc_accept_total = 0
    accepted_runs = set()  # Track which runs accepted the move in each cycle

    # Testing phase: process all MC runs in parallel for BIG_MOVE_ATTEMPTS cycles
    for cycle in range(BIG_MOVE_ATTEMPTS):
        # First, run displacement steps for all MC runs
        for run_idx, mc_run in enumerate(mc_runs):
            for step in range(1, BIG_MOVE_INTERVAL + 1):
                mc_run.particle_displacement()
                total_mcmc_steps += 1
                if total_mcmc_steps % TESTING_SAMPLING_FREQUENCY == 0:
                    sample = mc_run.sample(step)
                    mc_run.local_samples.append(sample)
                    mc_run.testing_samples.append(sample[6])  # store configuration (in MC box coordinates)
        
        # Reset accepted runs for this cycle
        accepted_runs.clear()
        
        # Make big move attempts for all MC runs
        for run_idx, mc_run in enumerate(mc_runs):
            # Each MC run receives a unique configuration for a big move suggestion
            test_config = test_configs[cycle * len(mc_runs) + run_idx]
            
            # Attempt the big move
            nf_to_mc_attempts_total += 1
            accepted = mc_run.nf_big_move(test_config)
            
            if accepted:
                nf_mc_accept_total += 1
                accepted_runs.add(run_idx)
                
                # Initialize testing move counters if they don't exist yet
                if not hasattr(mc_run, 'test_moves_accepted'):
                    mc_run.test_moves_attempted = 0
                    mc_run.test_moves_accepted = 0
                
                # Update testing counters for this simulation run
                mc_run.test_moves_attempted = mc_run.test_moves_attempted + 1 if hasattr(mc_run, 'test_moves_attempted') else 1
                mc_run.test_moves_accepted += 1
                
        
        # Calculate and log acceptance statistics for this cycle
        p_acc_current = nf_mc_accept_total / nf_to_mc_attempts_total if nf_to_mc_attempts_total > 0 else 0
        p_acc_history.append(p_acc_current)
        mcmc_steps_history.append(total_mcmc_steps)
        
        # Update global counters
        big_move_attempts = nf_to_mc_attempts_total
        big_move_accepts = nf_mc_accept_total
        
        experiment_logger.info(f"Cycle {cycle+1} complete: p_accept = {p_acc_current:.4f} ({nf_mc_accept_total}/{nf_to_mc_attempts_total})")

    # Plot the acceptance rate history
    acc_plot_path_svg, acc_plot_path_png = plot_acceptance_rate(
        p_acc_history, 
        directory, 
        x_values=mcmc_steps_history, 
        xlabel='MCMC Steps',
        base_filename="nf_acceptance_rate",
        color='C2'
    )
    experiment_logger.info(f"Acceptance rate plot saved to: {acc_plot_path_png}")
    
for run_idx, mc_run in enumerate(mc_runs):
    if mc_run.testing_samples:
        testing_configs = np.array(mc_run.testing_samples)  # shape: (num_samples, num_particles, 2)
        testing_configs = testing_configs[:]
        
        # Compute well statistics
        avg_x_values, p_a_values, p_b_values, deltaF_normalized_values, runs = calculate_well_statistics(
            testing_configs, 0, HALF_BOX, R0
        )

        # Add this run's free energy data to the array
        free_energy_array.append(deltaF_normalized_values)

        # Plot well statistics
        run_folder = os.path.join(mc_runs_directory, f"run_{run_idx+1:03d}")
        well_stats_path_svg, well_stats_path_png = plot_well_statistics(
            avg_x_values, 
            p_a_values, 
            p_b_values, 
            deltaF_normalized_values, 
            runs, 
            HALF_BOX,
            run_folder
        )
        
        # Plot average x coordinate for individual particles
        avg_x_path_svg, avg_x_path_png = plot_avg_x_coordinate(
            testing_configs,
            run_folder,
            HALF_BOX,
            run_idx+1
        )



# Plot summary of the first 10 MC runs if there are at least 10
if len(mc_runs) >= 10:
    multi_avg_x_path_svg, multi_avg_x_path_png = plot_multiple_avg_x_coordinates(
        mc_runs,
        directory
    )
    experiment_logger.info(f"Summary plot of first 10 MC runs saved to: {multi_avg_x_path_svg} and {multi_avg_x_path_png}")

# Plot average free energy across all runs
free_energy_plot_path_svg, free_energy_plot_path_png, final_mean, final_sem, final_std = plot_avg_free_energy(
    free_energy_array,
    directory,
    color='C2'
)
experiment_logger.info(f"Average free energy plot saved to: {free_energy_plot_path_svg} and {free_energy_plot_path_png}")
experiment_logger.info(f"Final mean delta F = {final_mean}")
experiment_logger.info(f"Final standard error delta F = {final_sem}")
experiment_logger.info(f"Final std delta F = {final_std}")

# -------------------------
# After the full experiment, save each MC run's sampled data to a CSV file and the configurations (sample[6] for each sample) are saved as an NPY file
# -------------------------
for i, mc_run in enumerate(mc_runs):
    run_folder = os.path.join(mc_runs_directory, f"run_{i+1:03d}")
    csv_filename = os.path.join(run_folder, 'sampled_data.csv')
    print(f"Saving sampled data to {csv_filename}")
    
    try:
        with open(csv_filename, 'w', newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow([
                "cycle_number",
                "energy_per_particle",
                "density",
                "pressure",
                "box_size_x",
                "box_size_y",
                "particle_configuration"
            ])
            
            for data in mc_run.local_samples:
                # Expecting that each sample is structured as:
                # (cycle_number, energy_per_particle, density, pressure, box_size_x, box_size_y, particles)
                cycle_number, energy_per_particle, density, pressure, box_size_x, box_size_y, particles = data
                
                # Convert the particle configuration to a NumPy array and flatten it.
                particles_flat = np.array(particles).flatten()
                
                writer.writerow([
                    cycle_number,
                    energy_per_particle,
                    density,
                    pressure,
                    box_size_x,
                    box_size_y,
                    particles_flat.tolist()  # storing as a JSON-like list in the CSV
                ])
        experiment_logger.info(f"MC run {i+1} sampled data successfully saved to {csv_filename}")
    except Exception as e:
        experiment_logger.error(f"Error: Failed to save sampled data for MC run {i+1} to {csv_filename}. Error: {e}")

    # Extract and save local samples configurations
    configs = np.array([sample[6] for sample in mc_run.local_samples])
    configs_filepath = os.path.join(run_folder, "mc_run_configs.npy")
    np.save(configs_filepath, configs)
    experiment_logger.info(f"MC run {i+1} local config data saved to: {configs_filepath}")

    # Extract and save testing samples configurations
    testing_configs = np.array(mc_run.testing_samples)
    testing_configs_filepath = os.path.join(run_folder, "mc_run_testing_configs.npy") 
    np.save(testing_configs_filepath, testing_configs)
    experiment_logger.info(f"MC run {i+1} testing config data saved to: {testing_configs_filepath}")
